## Units and Uncertainties in SpectroChemPy


In [1]:
from spectrochempy.api import *


        SpectroChemPy's API
        Version   : 0.1a1.3
        Copyright : 2014-2017, LCS - Laboratory for Catalysis and Spectrochempy
            


### Units
Units in SpectroChemPy are based on the `Pint` (library. Quantities with units can be created using the various following syntaxes.

In [2]:
19.4 * ur.cm

<Quantity(19.4, 'centimeter')>

In [3]:
19.4 * ur('cm')

<Quantity(19.4, 'centimeter')>

In [4]:
Quantity(19.4, ur.cm)

<Quantity(19.4, 'centimeter')>

In [5]:
Quantity(19.4, 'cm')

<Quantity(19.4, 'centimeter')>

In [6]:
Quantity('19.4 cm')

<Quantity(19.4, 'centimeter')>

Conversion between units are easy

In [7]:
a = Quantity('19.4 cm')
a

<Quantity(19.4, 'centimeter')>

In [8]:
a.ito(ur.inch)    # change a inplace
b = a.to(ur.km)   # change a to a new variable 
print(a)
print(b)

7.637795275590551 inch
0.000194 kilometer


Format strings can be used to format numeric output

In [9]:
b = 1/(a.to('in'))
print('{:.3fK}'.format(b))  # in latex format

0.131 in^-1


Dimensionless units are also handled:

In [10]:
b = a / (1.0 * ur.angstrom)
print('{:.3fK}'.format(b))
print('{:.3f}'.format(b))

7.638 scaled-dimensionless (2.5e+08)
7.638 inch / angstrom


Scaled dimensionless quantity can be made unscaled:

In [11]:
b.to_base_units()

<Quantity(1940000000.0, 'dimensionless')>

If an units is not recognized, an error is issued

In [12]:
wrong = 1.0 * ur('cmt')

 ERROR | UndefinedUnitError: 'cmt' is not defined in the unit registry


or if units cannot be converted:

In [13]:
a = Quantity('1 liter')
a.to('meters') # wrong conversion (units not compatible)

 ERROR | DimensionalityError: Cannot convert from 'liter' ([length] ** 3) to 'meter' ([length])


In [14]:
x = 10 * ur.meters
np.cos(x) # wrong argument units

 ERROR | DimensionalityError: Cannot convert from 'meter' to 'radian'


Spectroscopic units are silently converted between frequency and wavelength:

In [15]:
a = Quantity('1 THz')
a.ito('micrometer') 
a

<Quantity(299.792458, 'micrometer')>

The corresponding wavenumber is easily obtained

In [16]:
b = 1/a
b.ito('cm^-1')
print('{:.3fK}'.format(b))

33.356 cm^-1


Finally, the units can be affected to array structure

In [17]:
c = [3., 4., 5.] * ur.meter
c

<Quantity([       3        4        5], 'meter')>

In [18]:
c.ito('km')
c

<Quantity([   0.003    0.004    0.005], 'kilometer')>